https://adventofcode.com/2018/day/15

In [36]:
from dataclasses import dataclass
from collections import deque
from toolz import concat

In [4]:
with open('data/15-1.txt') as fh:
    data = fh.read()

In [5]:
print(data)

################################
#####################...########
###################....G########
###################....#########
#######.##########......########
#######G#########........#######
#######G#######.G.........######
#######.######..G.........######
#######.......##.G...G.G..######
########..##..#....G......G#####
############...#.....G.....#####
#...#######..........G.#...#####
#...#######...#####G......######
##...######..#######G.....#.##.#
###.G.#####.#########G.........#
###G..#####.#########.......#.E#
###..######.#########..........#
###.......#.#########.....E..E.#
#####G...#..#########.......#..#
####.G.#.#...#######.....G.....#
########......#####...........##
###########..................###
##########.................#####
##########.................#####
############..E.........E.....##
############.........E........##
###############.#............E##
##################...E..E..##.##
####################.#E..####.##
################.....######...##
##########

In [6]:
@dataclass(repr=True, order=True, frozen=True)
class Pt:
    row: int
    col: int

    @property
    def neighbors(self):
        """Sorted neighbor coords"""
        return [
            Pt(self.row - 1, self.col),
            Pt(self.row, self.col - 1),
            Pt(self.row, self.col + 1),
            Pt(self.row + 1, self.col)
        ]


@dataclass(repr=True)
class Fighter:
    team: str
    hp: int = 200
    ap: int = 3
    alive: bool = True
    
    
class GameOver(Exception):
    pass

In [8]:
def load_data(data):
    cave = set()
    fighters = {}
    for i, line in enumerate(data.split('\n')):
        for j, c in enumerate(line):
            if c in ('.', 'G', 'E'):
                pt = Pt(i, j)
                cave.add(pt)
                if c in ('G', 'E'):
                    fighters[pt] = Fighter(c)
   
    return cave, fighters


In [9]:
cave, fighters = load_data(data)

In [33]:
def shortest_path(node, goal, cave, fighters):
    q = deque([(node,)])
    visited = set()
    while q:
        pth = q.popleft()
        node = pth[-1]
        if node in visited:
            continue
        for nabe in node.neighbors:
            if nabe == goal:
                return pth + (nabe,)
            if nabe not in cave or nabe in fighters:
                continue
            q.append(pth + (nabe,))
        visited.add(node)

In [48]:
def turn(mypt, cave, fighters):
    print('.', end='')
    try:
        me = fighters[mypt]
    except KeyError: # already dead
        return

    enemy_team = 'G' if me.team == 'E' else 'E'
    enemies = {k:v for k, v in fighters.items() if v.team == enemy_team}
    if not enemies:
        raise GameOver
    
    def attack(foept):
        foe = fighters[foept]
        foe.hp -= me.ap
        if foe.hp <= 0:
            foe.alive = False
            del fighters[foept]
            del enemies[foept]

    attackable = [(enemies[nabe].hp, nabe) for nabe in mypt.neighbors if nabe in enemies and nabe in cave]        
    if attackable:
        _, foept = min(attackable)
        attack(foept)
        return
    
    targets = [x for x in set(concat(e.neighbors for e in enemies)) if x in cave and x not in fighters]
    startingpts = [x for x in mypt.neighbors if x in cave and x not in fighters]
    attack_paths = []
    for sp in startingpts:
        for tgt in targets:
            pth = shortest_path(sp, tgt, cave, fighters)
            if pth:
                attack_paths.append((len(pth), pth[-1], pth[0]))
    if not attack_paths:
        return
    
    _, _, moveto = min(attack_paths)
    
    del fighters[mypt]
    fighters[moveto] = me
    mypt = moveto

    attackable = [(enemies[nabe].hp, nabe) for nabe in mypt.neighbors if nabe in enemies and nabe in cave]        
    if attackable:
        _, foept = min(attackable)
        attack(foept)


In [39]:
def round(cave, fighters):
    for pt in sorted(fighters.keys()):
        turn(pt, cave, fighters)

In [ ]:
(cave, fighters) = load_data(data)

for i in range(1_000_000):
    try:
        round(cave, fighters)
    except GameOver:
        hps = sum(v.hp for v in fighters.values() if v.alive)
        print(i, hps, i * hps)
        break
else:
    print(i)

......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................